In [1]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("Python Spark SQL basic example") \
	    .config("spark.some.config.option", "some-value") \
	    .getOrCreate()
                            

ModuleNotFoundError: No module named 'pyspark'

In [3]:
parking_df = spark.read.csv(path='/shared/CS-GY-6513/parking-violations/parking-violations-header.csv',header=True)


In [4]:
parking_df.count()

1014017

In [5]:
parking_df.select('summons_number').distinct().count()

1014017

In [6]:
# Register the DataFrame as a SQL temporary view
parking_df.createOrReplaceTempView("parking")
spark.sql("SELECT count(DISTINCT plate_type) FROM parking").show()


+--------------------------+
|count(DISTINCT plate_type)|
+--------------------------+
|                        75|
+--------------------------+



In [7]:
# List the distinct plate types in the table


In [8]:
# List each distinct plate type and its frequency in the table
# e.g.
#    plate_type count
#    PAS        740554
#    COM        190147
    


In [9]:
# order the results of the previous query by count, the most frequent should appear first

In [10]:
# change plate_type 999 to null
from pyspark.sql import functions as F
parking_df2 = parking_df.withColumn('plate_type', F.when(parking_df['plate_type']=='999', 'null').otherwise(parking_df['plate_type']))
parking_df2.groupBy('plate_type').count().orderBy('count', ascending=False).show(26)


+----------+------+
|plate_type| count|
+----------+------+
|       PAS|740554|
|       COM|190147|
|       OMT| 35480|
|       OMS|  9032|
|       SRF|  8341|
|       IRP|  5291|
|      null|  4467|
|       TRC|  2784|
|       OMR|  2158|
|       APP|  1952|
|       MOT|  1851|
|       ORG|  1591|
|       CMB|  1368|
|       MED|  1211|
|       OML|  1181|
|       PSD|   900|
|       SPO|   823|
|       SCL|   700|
|       TOW|   611|
|       RGL|   524|
|       VAS|   427|
|       SRN|   348|
|       DLR|   333|
|       TRA|   318|
|       ITP|   283|
|       TRL|   223|
+----------+------+
only showing top 26 rows



In [11]:
# remove all rows where plate_type=999  



In [12]:
# Suppose we are interested in analyzing violations based on what county they occur.
# We might want to exclude rows that have a blank entry in the violation_county column.
# How many rows have Blank Entries in violation_county?
parking_df.filter( parking_df["violation_county"].isNull()).count()

192974

In [13]:
# Create a new dataframe without any black entries in violation_county
parking_df4 = 

SyntaxError: invalid syntax (4293702517.py, line 2)

In [14]:
# Let's clean the plate_ids
# “Clustering” helps detect entries in a column that are close together (and thus represent the same value

plate_id_rdd = parking_df.select('plate_id').rdd.flatMap(list)
plate_id_rdd.take(50)

['GBH2444',
 'GKZ2313',
 'N346594',
 'GDP2624',
 '42555JU',
 '62636MD',
 'DPE3045',
 'FMW7832',
 'DSD2130',
 '65111MB',
 'GMZ3750',
 '44884',
 'XZ876G',
 'PIKINE',
 'GMU4296',
 'GEJ8235',
 '74452JW',
 '42972JW',
 '66951',
 '63400JM',
 'GGS5172',
 '51329A',
 '49216KA',
 '31695JZ',
 '79638KA',
 '88720MB',
 'ERP5344',
 'FWM1758',
 '14307LV',
 'EWT1353',
 '65566PA',
 'FPF5158',
 '24393MC',
 '24393MG',
 'FXR1798',
 'FWH9893',
 '88629JH',
 '1510332',
 'DJE1615',
 '2208656',
 'GXC7520',
 'GRC4443',
 'T639084C',
 'GARFR5',
 'GTJ6780',
 '401ZGU',
 'ZWF21Z',
 'S274036',
 'PF090W',
 'GLR6718']

"Key Collision" methods are based on the idea of creating a key value that contains only 
 the most valuable or meaningful part of the string and groups together different strings based 
 on the fact that their key is the same (hence the name "key collision").
Fingerprinting Method:
note that the order of these operations (the last 3 lines) is significant.
remove leading and trailing whitespace	
change all characters to their lowercase representation
remove all punctuation and control characters
normalize extended western characters to their ASCII representation (for example "gödel" → "godel")
split the string into whitespace-separated tokens
sort the tokens and remove duplicates
join the tokens back together


In [15]:
import string, unicodedata
def fingerprint(value):
    key = unicodedata.normalize('NFKD', value).encode('ascii','ignore').decode()
    key = set(key.strip().lower().translate(str.maketrans('','',string.punctuation)).split())
    key = ' '.join(sorted(list(key)))
    return (key, value)

In [16]:
plate_id_rdd.distinct().map(fingerprint).first()

('gkz2313', 'GKZ2313')

In [ ]:
# apply the fingerprint function to all plate_id values, and group the ones
# that have the same key
# mapValues(list) - makes the grouped values into a list
plate_id_rdd.distinct(). \
	map(fingerprint). \
	groupByKey(). \
	filter(lambda x: len(x[1])>1). \
	mapValues(list). \
	collect()


In [22]:
plate_id_rdd.distinct(). \
	map(fingerprint). \
	groupByKey(). \
	filter(lambda x: len(x[1])>1).take(5)


[('2970cp', <pyspark.resultiterable.ResultIterable at 0x7f31d918a6a0>),
 ('ap717y', <pyspark.resultiterable.ResultIterable at 0x7f31d918adf0>),
 ('xbgv20', <pyspark.resultiterable.ResultIterable at 0x7f31d918ad60>),
 ('849rzb', <pyspark.resultiterable.ResultIterable at 0x7f31d9185820>),
 ('ab73725', <pyspark.resultiterable.ResultIterable at 0x7f31d9185130>)]

In [ ]:
# To determine a cluster should be merged or not, you can look more closely at the data.
# check the other attributes to determine if the two ids correspond to the same entity
parking_df.where((parking_df.plate_id == '2970CP') | \
	(parking_df.plate_id == '2970.CP')). \
	show()


In [ ]:
# clustering on the street_name column.
street_name_rdd = parking_df.select('street_name').distinct().rdd.flatMap(list).filter(lambda x: x!=None)

stname_clusters = 

In [ ]:
street_name_rdd.take(3)

In [ ]:
# apply the fingerprint to all street addresses
#[('court square', 'COURT SQUARE'),
# ('31 e st', 'E 31 ST'),
# ('island randalls', 'RANDALLS ISLAND'),
# ('45th e street', 'E 45TH STREET'),
# ('ave sheridan', 'SHERIDAN AVE')]